必需要的資料:

*熱門遊戲種類
*1."種類名稱" 2."遊戲名稱" 3."所屬公司" 4."發行日期" 5."評分" 6."遊戲介紹"

*遊戲新聞
*1."標題名稱" 2."時間" 3."地點" 4."新聞內容"

*展覽資訊
*1."主題名稱" 2."時間" 3."地點" 4."展覽內容"

In [1]:
def open_db():#開啟Database
    import pymysql.cursors
    db = pymysql.connect(host='deepbowl-mysql.cukfiecfgfog.ap-northeast-1.rds.amazonaws.com',#連結MySQL
                             user='public_root',
                             password='thisisasamplepassword',                             
                             db='game_chatbot',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
    return db

In [2]:
def search_db(table=None, columns = None, limit = None,operation = None, keyword = None, quantity=None):
    #talbe:資料表可能會有Join在一起的表，不過這是用字串儲存
    #columns:欄位的名稱
    #limit:篩選條件,Ex:game_name,game_type等
    #operation:操做,Ex:like,= 等
    #keyword:要查的關鍵字,Ex:遊戲名等
    
    db = open_db()
    try:
        cursor = db.cursor()#建立資料庫游標
        sql = ("select "+columns+" \
                from "+table+" \
                where "+limit+" "+operation+" '"+keyword+"' ")#下指令，皆用變數儲存
        
        cursor.execute(sql)
        rows = cursor.fetchall()
        return rows
    except Exception as e:
        print("Exeception occured:{}".format(e))
    finally:
        db.close()#關閉DataBase

In [3]:
def game_type(game_type = None):#查詢遊戲類型
    import random#亂數
    table = 'type inner join (game inner join game_has_type using (game_ID)) using (type_ID)'#目標Table
    columns = 'distinct(game_name)'#詢問遊戲類型時有的columns:遊戲名稱
    limit = 'type_name'
    operation = 'like'
    game_type = '%'.join(i for i in game_type)#字與字之間插入，SQL like語法要用的'%'符號
    game_type = '%' + game_type + '%'#頭跟尾加上SQL like語法要用的'%'符號
    ans = search_db(table,columns, limit,operation, keyword = game_type)#進資料庫搜尋的各項參數設定
    if ans:
        random.shuffle(ans)#打亂查詢到遊戲名稱的list
        ans = '\u000A'.join(i['game_name'] for i in ans[0:5])#取前5項的遊戲名稱並用'，'合併
    else:
        ans = 'haha none'
    return ans

In [4]:
def game_introduction(game_name = None):#遊戲介紹的Funtion
    table = 'game'#目標Table
    columns = 'game_description,game_release_date'#介紹遊戲時有的columns:介紹內容、發行日期
    limit = 'game_name'
    operation = 'like'
    game_name = '%'.join(i for i in game_name)#字與字之間插入，SQL like語法要用的'%'符號
    game_name = '%' + game_name + '%'#頭跟尾加上SQL like語法要用的'%'符號
    ans = search_db(table,columns, limit, operation,keyword = game_name)#進資料庫搜尋的各項參數設定
    if ans:
        ans = str(ans[0]['game_description']) + 'Release Date: ' + ans[0]['game_release_date'].strftime('%Y/%m/%d')#合併columns，DateTime類型轉String使用strftime
    else:
        ans = 'haha none'
    return ans

In [5]:
def game_news(game_name = None):#遊戲新聞的Funtion
    table = 'game inner join(news_has_game inner join news using (news_ID)) using (game_Id)'#game,news_has_game,news三個Table Inner Join在一起
    columns = 'game_name,news_topic,news_part,news_URL'#
    limit = 'game_name'
    operation = 'like'
    game_name = '%'.join(i for i in game_name)#字與字之間插入，SQL like語法要用的'%'符號
    game_name = '%' + game_name + '%'#頭跟尾加上SQL like語法要用的'%'符號
    ans = search_db(table,columns, limit, operation,keyword = game_name)#進資料庫搜尋的各項參數設定
    if ans:
        ans = ans[0]['game_name'] + '\u000A' + ans[0]['news_topic'] + '\u000A' + ans[0]['news_part'] + '\u000A' + ans[0]['news_URL']#合併columns，'\u000A'為傳輸到Line的換行操作字元
    else:
        ans = 'haha none'
    return ans

In [6]:
def game_price(game_name = None):#遊戲價錢的Funtion
    table = 'game'
    columns = 'game_price,game_price_currency'
    limit = 'game_name'
    game_name = '%'.join(i for i in game_name)#字與字之間插入，SQL like語法要用的'%'符號
    game_name = '%' + game_name + '%'#頭跟尾加上SQL like語法要用的'%'符號
    operation = 'like'
    
    ans = search_db(table,columns, limit,operation,keyword = game_name)
    if ans:
        ans = game_name + '的價錢是: ' +str(ans[0]['game_price']) + ans[0]['game_price_currency']
    else:
        ans = 'haha none'
    return ans

In [7]:
def game_recommed():#遊戲推薦的Funtion
    import random #亂數
    table = 'game'
    columns = 'game_name,game_score'
    keyword = '壓倒性好評'
    limit = 'game_score'
    operation = '='
    ans = search_db(table,columns,limit,operation,keyword)
    if ans:
        random.shuffle(ans)#打亂list順序
        ans = ('\u000A'.join(i['game_name'] + ' 網路評價:' + i['game_score'] for i in ans[0:5]))#合併columns，'\u000A'為傳輸到Line的換行操作字元
    else:
        ans = 'haha none'
    return ans

In [17]:
#未完成
def game_Time(checkDict):#遊戲推薦的Funtion
    import datetime
    table = ''
    columns = 'game_release_date'
    for i in checkDict:
        if i == 'any' and checkDict[i] != '':
            table.join('game')
            columns.join('game_name')
        elif i == 'game_date' and checkDict[i] != '':
            columns.join(',game_release_date')
        elif i == 'timenumber' and checkDict[i] != '':


In [18]:
import json
from flask import Flask, request, make_response, jsonify
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pymysql.cursors

app = Flask(__name__)


@app.route("/")
def verify():
    return "Hello world", 200


@app.route('/', methods=['POST'])#Dialogflow端
def webhook():
    req = request.get_json(silent=True, force=True)
    print(req)
    retJson = ''#查詢完的結果Asign到這兒

    
    if req['queryResult']['intent']['displayName'] != None:#確認Intent是否存在
        targetIntent = req['queryResult']['intent']['displayName']#儲存Intent的名稱  
        
    if targetIntent == 'askGameIntroduction':#Intent的名稱是遊戲介紹
        if req['queryResult']['parameters']['game_introduction'] != None:
            keyword = req['queryResult']['parameters']['any']
            retJson = game_introduction(game_name = str(keyword))
            
    if targetIntent == 'askGameType':#Intent的名稱是遊戲類型
        if req['queryResult']['parameters']['typeflag'] != None:
            keyword = req['queryResult']['parameters']['any']
            retJson = game_type(game_type = str(keyword))
            
    if targetIntent == 'askGameRecommend':#Intent的名稱是遊戲類型
        if req['queryResult']['parameters']['game_recommend'] != None:
            retJson = game_recommed()
            
    if targetIntent == 'askGameNews':#Intent的名稱是遊戲新聞
        if req['queryResult']['parameters']['game_news'] != None:
            keyword = req['queryResult']['parameters']['any']
            retJson = game_news(game_name = keyword)
            
    if targetIntent == 'askGamePrice':#Intent的名稱是遊戲價錢
        if req['queryResult']['parameters']['game_price'] != None:
            keyword = req['queryResult']['parameters']['any']
            retJson = game_news(game_name = keyword)

#     if targetIntent == 'askGameTime': #Intent的名稱是遊戲時間
#         checkDict =  req['queryResult']['parameters']:
#         retJson = game_Time(checkDict)
        
    res = {"fulfillmentText": retJson}#回傳DialogFlow的Json
    res = json.dumps(res,indent = 4)
    print(res)
    return res

# @app.route('/lastgame', methods=['GET'])#web端
# def webView():
#     cursor = db.cursor()#建立資料庫游標
    
#     sql = ("select * from game")#下指令
    
#     cursor.execute(sql)
#     rows = cursor.fetchall()
    
    
#     req = request.get_json(silent=True, force=True)
#     print(req)
    
#     return res

if __name__ == '__main__':
    app.run(port=5000)
    

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Dec/2018 15:31:49] "POST / HTTP/1.1" 200 -


{'responseId': '445acd06-78cd-45bb-84fe-422b471dcd69', 'queryResult': {'queryText': '半年內的遊戲', 'parameters': {'any': '', 'game_date': '', 'typeflag': '', 'timenumber': '0.5', 'timedmy': ['年'], 'timetf': ['-']}, 'allRequiredParamsPresent': True, 'fulfillmentText': '這段時間沒有遊戲發行', 'fulfillmentMessages': [{'text': {'text': ['這段時間沒有遊戲發行']}, 'platform': 'LINE'}, {'text': {'text': ['這段時間沒有遊戲發行']}}], 'intent': {'name': 'projects/newagent-bc844/agent/intents/03444649-2782-430b-8c21-969ca2eb8631', 'displayName': 'askGameTime'}, 'intentDetectionConfidence': 1.0, 'languageCode': 'zh-cn'}, 'originalDetectIntentRequest': {'payload': {}}, 'session': 'projects/newagent-bc844/agent/sessions/2ff2ac85-aee9-c165-adc7-5de790691dd0'}
1 

{'any': 0, 'game_date': 0, 'typeflag': 0, 'timenumber': 1, 'timedmy': 1, 'timetf': 1}
{
    "fulfillmentText": null
}


In [19]:
import datetime
print(datetime.datetime.now())

2018-12-25 16:23:44.089593
